In [ ]:
num_classes = 5


: 

In [ ]:
def VGG16_1D(input_shape , n_classes=5):
    model = Sequential()

    # First block
    model.add(Conv1D(filters=64, kernel_size=3, strides=1, padding='same', input_shape=input_shape, activation='relu'))
    model.add(Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))  # Added padding='same' to avoid shrinking

    # Second block
    model.add(Conv1D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
    model.add(Conv1D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))  # Added padding='same'

    # Third block
    model.add(Conv1D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'))
    model.add(Conv1D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'))
    model.add(Conv1D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))  # Added padding='same'

    # Flatten before Dense layers
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(units=4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=4096, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer (adjust for the number of classes)
    model.add(Dense(units= n_classes  , activation='softmax'))

    return model


In [ ]:
def AlexNet_1D(input_shape , n_classes= 5 ):
    model = Sequential()

    # First block
    model.add(Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))  # Add padding='same' to avoid shrinking

    # Second block
    model.add(Conv1D(filters=192, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))  # Add padding='same'

    # Third block

    model.add(Conv1D(filters=384, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv1D(filters=256, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))  # Add padding='same'

    # Flatten before Dense layers
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense( units =  n_classes , activation='softmax'))

    return model

In [ ]:
def VGG19_1D(input_shape , n_classes=5):
    input_layer = Input(shape=input_shape)

    # Block 1
    x = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(input_layer)
    x = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(x)
    x = MaxPooling1D(pool_size=2, strides=2, padding='same')(x)

    # Block 2
    x = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
    x = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
    x = MaxPooling1D(pool_size=2, strides=2, padding='same')(x)

    # Block 3
    x = Conv1D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
    x = Conv1D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
    x = Conv1D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
    x = MaxPooling1D(pool_size=2, strides=2, padding='same')(x)

    # Block 4
    x = Conv1D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
    x = Conv1D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
    x = Conv1D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
    x = MaxPooling1D(pool_size=2, strides=2, padding='same')(x)

    # Block 5
    x = Conv1D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
    x = Conv1D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
    x = Conv1D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
    x = MaxPooling1D(pool_size=2, strides=2, padding='same')(x)

    # Block 6 - Fully connected layers
    x = Flatten()(x)
    x = Dense(units=4096, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    x = Dense(units=4096, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    x = Dense(units=n_classes , activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=x, name='VGG19_1D')
    return model


In [ ]:
# Define 1D Convolutional Block
def conv_block_1d(x, filters, kernel_size, strides, padding='same'):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

# Identity Block for ResNet
def identity_block_1d(x, filters):
    shortcut = x
    x = conv_block_1d(x, filters=filters, kernel_size=1, strides=1)
    x = conv_block_1d(x, filters=filters, kernel_size=3, strides=1)
    x = Conv1D(filters=filters * 4, kernel_size=1)(x)
    x = BatchNormalization()(x)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

# Projection Block for ResNet
def projection_block_1d(x, filters, strides):
    shortcut = x
    x = conv_block_1d(x, filters=filters, kernel_size=1, strides=strides)
    x = conv_block_1d(x, filters=filters, kernel_size=3, strides=1)
    x = Conv1D(filters=filters * 4, kernel_size=1)(x)
    x = BatchNormalization()(x)
    shortcut = Conv1D(filters=filters * 4, kernel_size=1, strides=strides)(shortcut)
    shortcut = BatchNormalization()(shortcut)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

# Define ResNet50 1D
def ResNet50_1D(input_shape , n_classes=5 ):
    inputs = Input(shape=input_shape)

    # Initial conv layer
    x = conv_block_1d(inputs, filters=64, kernel_size=7, strides=2, padding='same')
    x = MaxPooling1D(pool_size=3, strides=2, padding='same')(x)

    # Conv block 1
    x = projection_block_1d(x, filters=64, strides=1)
    x = identity_block_1d(x, filters=64)
    x = identity_block_1d(x, filters=64)

    # Conv block 2
    x = projection_block_1d(x, filters=128, strides=2)
    x = identity_block_1d(x, filters=128)
    x = identity_block_1d(x, filters=128)
    x = identity_block_1d(x, filters=128)

    # Conv block 3
    x = projection_block_1d(x, filters=256, strides=2)
    x = identity_block_1d(x, filters=256)
    x = identity_block_1d(x, filters=256)
    x = identity_block_1d(x, filters=256)
    x = identity_block_1d(x, filters=256)
    x = identity_block_1d(x, filters=256)

    # Conv block 4
    x = projection_block_1d(x, filters=512, strides=2)
    x = identity_block_1d(x, filters=512)
    x = identity_block_1d(x, filters=512)

    # Global average pooling and dense layer
    x = GlobalAveragePooling1D()(x)
    outputs = Dense( units=n_classes  , activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# Define the MobileNet 1D model
def mobilenet_1d(input_shape , n_classes):
    # Input layer
    inputs = tf.keras.Input(shape=input_shape)

    # Convolutional layers
    x = layers.Conv1D(32, 3, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Depthwise separable convolutions
    x = layers.SeparableConv1D(64, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.SeparableConv1D(128, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.SeparableConv1D(128, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.SeparableConv1D(256, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.SeparableConv1D(256, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.SeparableConv1D(512, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    for _ in range(5):
        x = layers.SeparableConv1D(512, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

    x = layers.SeparableConv1D(1024, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Global average pooling
    x = layers.GlobalAveragePooling1D()(x)

    # Output layer
    outputs = layers.Dense( units=n_classes  , activation='softmax')(x)

    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model